In [378]:
#initial imports
import requests
import pandas as pd
import xlwings as xw
import time as tm
import yfinance as yfimport 
from datetime import datetime


url = "https://www.nseindia.com/api/option-chain-indices?symbol=NIFTY"

headers = {"Accept-Encoding":"gzip, deflate, br",
           "Accept-Language":"en-US,en;q=0.9",
           "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (HTML, like Gecko) Chrome/116.0.0.0 Safari/537.36"
}

session = requests.Session()
data  = session.get(url,headers=headers).json()["records"]["data"]

ocdata = []

for i in data:
    for j,k in i.items():
        if j == "CE" or j == 'PE':
            info = k
            info["instrument_type"] = j
            ocdata.append(info)


In [379]:
#live nifty price
nifty_ticker = yf.Ticker("^NSEI")
current_price = nifty_ticker.history(period = "id")["Close"].iloc[0]
current_price


19674.25

In [380]:
#creating Data dataframe and sorting accourding to date 
oc = pd.DataFrame(ocdata)
oc_sorted = oc.sort_values(by='expiryDate')


In [381]:
#Making copy of original sorted df and sort ce and pe
df = oc_sorted.copy()
group = df.groupby('instrument_type')
ce_group = group.get_group('CE')
pe_group = group.get_group('PE')
#required columns
CE_col = ['openInterest','changeinOpenInterest','totalTradedVolume','lastPrice']
PE_col = ['strikePrice','lastPrice','totalTradedVolume','changeinOpenInterest','openInterest','expiryDate']
ce_group = ce_group[CE_col]
pe_group = pe_group[PE_col]
a = df[CE_col]
b = df[PE_col]
a = a.rename(columns={'openInterest': 'ce_openInterest', 'changeinOpenInterest': 'ce_changeinOpenInterest', 'totalTradedVolume': 'ce_totalTradedVolume', 'lastPrice': 'ce_lastPrice'})
b = b.rename(columns={'openInterest': 'pe_openInterest', 'changeinOpenInterest': 'pe_changeinOpenInterest', 'totalTradedVolume': 'pe_totalTradedVolume', 'lastPrice': 'pe_lastPrice'})
df1 = pd.concat([a,b],axis=1)



In [382]:
#making proper date format
uni = df1['expiryDate'].unique()
df1['expiryDate'] = pd.to_datetime(df1['expiryDate'],format='%d-%b-%Y')
nearest_expiry = df1['expiryDate'].min()
#nearest expiry
formatted_date = nearest_expiry.strftime('%d-%b-%Y')
present_week_expiry= formatted_date
present_week_expiry


'28-Sep-2023'

In [383]:
#finding the nearest expiry
df1 = df1[df1['expiryDate'] == present_week_expiry]
df1 = df1.sort_values(by='strikePrice',ascending=True)
df1

,ce_openInterest,ce_changeinOpenInterest,ce_totalTradedVolume,ce_lastPrice,strikePrice,pe_lastPrice,pe_totalTradedVolume,pe_changeinOpenInterest,pe_openInterest,expiryDate
2,8759.0,7396,70236,0.40,12000,0.40,70236,7396,8759.0,2023-09-28
3,4.0,2,4,7729.80,12000,7729.80,4,2,4.0,2023-09-28
8,8256.0,-599,614,6710.60,13000,6710.60,614,-599,8256.0,2023-09-28
7,5812.0,54,24193,0.45,13000,0.45,24193,54,5812.0,2023-09-28
21,3131.0,1710,15632,0.40,14000,0.40,15632,1710,3131.0,2023-09-28
...,...,...,...,...,...,...,...,...,...,...
1137,8156.0,-221,756,2286.00,22000,2286.00,756,-221,8156.0,2023-09-28
1158,26583.0,846,86291,1.05,23000,1.05,86291,846,26583.0,2023-09-28
1157,2708.0,-319,455,3289.00,23000,3289.00,455,-319,2708.0,2023-09-28
1174,30627.0,5842,278737,0.75,24000,0.75,278737,5842,30627.0,2023-09-28


In [384]:
#To find ATM(at the money)
target_value = current_price

df1['difference'] = abs(df1['strikePrice']- )
diff = df1.loc[df1['difference'].idxmin()]
d = diff['strikePrice']

atm = df1[df1['strikePrice'] == d]
atm


SyntaxError: invalid syntax (119059491.py, line 4)